In [4]:
import requests
from bs4 import BeautifulSoup
import hashlib
import json
import random
import datetime
import bcrypt
import hashlib
minus = 'abcdefghijklmnñopqrsuvwxyz'
mayus = minus.upper()
numeros = "0123456789"
simbolos = "@()[]{}*,:./-_¿?¡!$%&'|#ºª´=+<>"


def extraer_libros(url):

  #Esta función extrae información de libros de un sitio web y la guarda en un archivo JSON.

      cabeceras = {'User-Agent': 'Mi aplicación Python'}

      try:
    # Realiza una solicitud GET a la URL especificada con los encabezados definidos
        respuesta = requests.get(url, headers=cabeceras)
        respuesta.raise_for_status()  # Genera una excepción para códigos de estado distintos a 200

    # Parsea el contenido HTML de la respuesta
        soup = BeautifulSoup(respuesta.text, 'html.parser')
    
        # Busca todos los elementos div
        lista_libros = soup.find_all('div', style="margin-top:5px")
    
        # Lista para almacenar los libros extraídos
        libros_extraidos = []
    
        for libro in lista_libros:
          # Extrae el título del libro (ajuste la clase del tag según el sitio web)
          titulo = libro.find('a', class_="primary-text-color").text.strip()
          # Extrae el autor del libro (ajuste la clase del tag según el sitio web)
          autor = libro.find('span', class_="book-description-author secondary-text-color").text.strip()
    
          # Validación opcional de los datos extraídos
          if titulo and autor:  # Comprueba si se encontraron el título y el autor
            libros_extraidos.append({'Título': titulo, 'Autor': autor, 'cantidad':5, 'prestado':0})
          else:
            print(f"Aviso: Se omite libro con datos faltantes (Título: {titulo}, Autor: {autor})")
    
        # Guarda los datos extraídos en un archivo JSON
        with open('Libros.json', 'w') as archivo:
          json.dump(libros_extraidos, archivo, indent=4)
    
    
      except requests.exceptions.RequestException as e:
        print(f"Error al realizar la solicitud: {e}")
      except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
    
if __name__ == "__main__":
  # Define la URL del sitio web que contiene la lista de libros
  url = 'https://www.elejandria.com/'
  # Inicia el proceso de extracción de libros
  extraer_libros(url)



class Biblioteca:
    Libros = "Libros.json"
    Usuarios = "usuarios.json"   #<--Inicializar las variables con los archivos para utilizarlos después
    reservas = "reservas.json"
    def __init__(self, archivo_libros="Libros.json", archivo_reservas="reservas.json"):
        self.libros = self.cargar_libros(archivo_libros)
        self.seguridad = Seguridad()
        self.usuarios = self.cargar_usuarios(archivo_reservas)
        self.reservas = []

    def cargar_libros(self, archivo_libros): #Cargar los libros del json que tiene el scraping de la web
        try:
            with open(archivo_libros, 'r') as f:  #<-- cargo la información del json
                libros = json.load(f)
                return libros
        except FileNotFoundError:
            print(f"El archivo {archivo_libros} no se encontró.")
            return []
        except json.JSONDecodeError:
            print("Error al decodificar el archivo JSON.")
            return []

    def cargar_usuarios(self, archivo_usuarios):#Cargar usuarios del json
    
            try:
                with open("usuarios.json", 'r') as f:
                    usuarios = json.load(f)
                return usuarios
            except FileNotFoundError:
                print(f"El archivo de usuarios {usuarios.json} no se encontró.")
                return []
            except json.JSONDecodeError:
                print("Error al decodificar el archivo JSON de usuarios.")
                return []
    
    def iniciar_sesion(self): #Incio de sesión
        usuario = input("Ingrese su nombre de usuario: ")
        contraseña = input("Ingrese su contraseña: ")
    
        usuarios = self.cargar_usuarios("usuarios.json")
        for user in usuarios:
            if user["usuario"] == usuario:
                hashed_password_almacenada = user["contraseña_hash"]
    
                # Generar el hash de la contraseña ingresada
                hashed_password_ingresada = hashlib.sha256(contraseña.encode('utf-8')).hexdigest()
    
                # Comparar los hashes
                if hashed_password_ingresada == hashed_password_almacenada:
                    print("Inicio de sesión exitoso.")
                    self.menu_principal(usuario)
                    return True
                else:
                    print("Contraseña incorrecta.")
                    return False
        return False



    
    def registrar_usuario(self):
    
        usuario = input("Ingrese su nombre de usuario: ")
        peticion = input("1-->Si quieres generar una contraseña\n 2-->Si prefieres introducir una contraseña: ")
        if peticion == "1":
            contraseña = generar_contraseña()
            print(contraseña)
            hashed_password = hashlib.sha256(contraseña.encode('utf-8')).hexdigest() #<-- cifro la contraseña del usuario nuevo
            hora_registro = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

                    # Crear un diccionario con los datos del usuario
            nuevo_usuario = {
                "usuario": usuario,
                "contraseña_hash": hashed_password,
                "hora_registro": hora_registro} 
    
            try:
                with open('usuarios.json', 'r') as f:  #<--cargar los usuarios existentes desde el archivo JSON (si existe)
                    usuarios = json.load(f)
            except FileNotFoundError:
                    usuarios = []
            usuarios.append(nuevo_usuario)
            
            with open('usuarios.json', 'w') as f:
                json.dump(usuarios, f, indent=2)   #<-- guardar la información en el Json

            print("Usuario registrado exitosamente")
            
        else:
            intentos = 0  # Inicializar el contador de intentos
            max_intentos = 3
            while intentos < max_intentos:
                contraseña = input("Ingrese su contraseña: ")
                if seguridad.comprobar_contraseña(contraseña):
                    print("La contraseña es segura")
                    hashed_password = hashlib.sha256(contraseña.encode('utf-8')).hexdigest()
                    hora_registro = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

                    # Crear un diccionario con los datos del usuario
                    nuevo_usuario = {
                        "usuario": usuario,
                        "contraseña_hash": hashed_password,
                        "hora_registro": hora_registro}
        
                    # Cargar los usuarios existentes desde el archivo JSON (si existe)
                    try:
                        with open('usuarios.json', 'r') as f:
                            usuarios = json.load(f)
                    except FileNotFoundError:
                        usuarios = []
    
                    usuarios.append(nuevo_usuario)
                    
                    with open('usuarios.json', 'w') as f:
                        json.dump(usuarios, f, indent=2)
        
                    print("Usuario registrado exitosamente")
                    break
                    
                else:
                    print("La contraseña no es segura, repitelo de nuevo")
                    intentos += 1
            if intentos == max_intentos:
                print('Has agotado los intentos')


    def buscar_usuario(self, usuario):
        try:
            with open('usuarios.json', 'r') as archivo:
                usuarios = json.load(archivo)  #<--cargar los usuarios existentes desde el archivo JSON (si existe)
    
            for user in self.usuarios:
                if user['usuario'] == usuario:
                    return user
    
        except FileNotFoundError:
            print("El archivo de usuarios no existe.")
        except json.JSONDecodeError:
            print("Error al cargar el archivo JSON.")
    
        return None
    
    
    
    def buscar_libro(self, titulo, autor):  #<-- Busco los títulos y autores de los libros
        resultados = []
        for libro in self.libros:
            if (titulo is None or titulo.lower() in libro.get('Título', '').lower()) and \
               (autor is None or autor.lower() in libro.get('autor', '').lower()):
                resultados.append(libro)
            else:
                print(f"El libro {libro} no coincide con los criterios de búsqueda.")
        return resultados
    
    

    def reservar_libro(self, usuario, titulo):  #<--Reservar libro
        libros_encontrados = self.buscar_libro(titulo, autor=None)
        if libros_encontrados:
            libro = libros_encontrados[0]
            if isinstance(libro, dict):
                cantidad = libro.get('cantidad', 0)
                prestado = libro.get('prestado', 0)
                if cantidad - prestado > 0:
                    if not self.usuario_tiene_reserva(usuario, titulo):
                        nueva_reserva = {
                            'usuario': usuario,
                            'libro': titulo,
                            'fecha_reserva': datetime.datetime.now().strftime("%Y-%m-%d"),
                            'fecha_devolucion': self.calcular_fecha_devolucion(datetime.datetime.now().strftime("%Y-%m-%d"))
                        }
                        self.reservas.append(nueva_reserva)
                        prestado += 1# Actualizar cantidad prestada
                        cantidad -=1
                        self.guardar_reservas()
                        self.guardar_cambios_en_json()
                        return "Reserva realizada con éxito"
                else:
                    return "El usuario ya tiene una reserva para este libro"
            else:
                return "No hay ejemplares disponibles"
        else:
            return "Libro no encontrado"
     
    
    def guardar_cambios_en_json(self):
        try:
            with open('libros.json', 'w') as f:
                json.dump(self.libros, f, indent=4)
        except IOError as e:
            print(f"Error al guardar los cambios: {e}")

    #Comprobar si el libro está disponible
    def libro_disponible(self, libro):
        if libro["cantidad"] > 0 and not libro["prestado"]:
            return True
        return False

    def usuario_tiene_reserva(self, usuario, titulo):
        # Implementa la lógica para verificar si un usuario ya tiene una reserva para un libro
        for reserva in self.reservas:
            if reserva['usuario'] == usuario and reserva['libro'] == titulo:
                return True
        return False    

    #guardar la reserva
    def guardar_reservas(self):
        try:
            with open('reservas.json', 'w') as f:
                json.dump(self.reservas, f, indent=4)
            print("Reservas guardadas correctamente.")
        except IOError:
            print("Error al guardar las reservas.")


    def validar_fecha(self, fecha):
        try:
            datetime.datetime.strptime(fecha, '%Y-%m-%d')
            return True
        except ValueError:
            return False
    #Calcular la fecha de devolución
    def calcular_fecha_devolucion(self, fecha_reserva, plazo=15):
        fecha_reserva_objeto = datetime.datetime.strptime(fecha_reserva, '%Y-%m-%d')
        fecha_devolucion = fecha_reserva_objeto + datetime.timedelta(days=plazo)
        return fecha_devolucion.strftime('%Y-%m-%d')

    # Mostrar las reservas
    def ver_mis_reservas(self, usuario, fecha_inicio=None, fecha_fin=None, ordenar_por='fecha', ascendente=True):
        try:
            with open('reservas.json', 'r') as f:
                reservas = json.load(f)

            mis_reservas = [reserva for reserva in reservas if reserva['usuario'] == usuario]
    
            if mis_reservas:
                print(f"Reservas de {usuario}:")
                for reserva in mis_reservas:
                    fecha_reserva_objeto = datetime.datetime.strptime(reserva['fecha_reserva'], '%Y-%m-%d')
                    try:
                        fecha_devolucion_objeto = datetime.datetime.strptime(reserva['fecha_devolucion'], '%Y-%m-%d')
                        print(f"- Libro: {reserva['libro']}")
                        print(f"  Fecha de reserva: {fecha_reserva_objeto.strftime('%d de %B de %Y')}")
                        print(f"  Fecha de devolución: {fecha_devolucion_objeto.strftime('%d de %B de %Y')}")
                        print("Si se pasa de la fecha de devolución, se le aplicará una multa de 10 euros. Gracias")
                    except KeyError:
                        print(f"La reserva para el libro {reserva['libro']} no tiene una fecha de devolución.")

        except FileNotFoundError:
            print("El archivo 'reservas.json' no existe.")
        except json.JSONDecodeError:
            print("Error al leer el archivo JSON. Asegúrate de que esté bien formateado.")
        except ValueError:
            print("Formato de fecha inválido. Utiliza YYYY-MM-DD.")


    def anular_reserva(self, usuario, titulo_libro):
        with open("reservas.json", "r") as f:
            reservas = json.load(f)
        for i, reserva in enumerate(reservas):
            if reserva['usuario'] == usuario and reserva['libro'] == titulo_libro:
                reservas.pop(i)
                break
        with open("reservas.json", "w") as f:
            json.dump(reservas, f, indent=4)
            print('Se ha anulado la reserva del libro:',titulo_libro)
            
    
    
    def menu_biblioteca(self):
        print('Biblioteca Los Hiperbóreos,', '¿En que podemos ayudarle?')
        print('1--> Ver libros disponibles')
        print('2--> Inicio de Sesión:')
        print('3--> Registrarse')
        print('4--> Salir')


    

    def eleccion_menu(self):
        while True:
            biblioteca.menu_biblioteca()
            opcion = input("Seleccione una opción: ")
            if opcion == "1":
                with open('Libros.json', 'r') as f:
                    data = json.load(f)
                    for libro in data:
                        print(f"Título: {libro['Título']}")
                        print(f"Autor: {libro['Autor']}")
                        print("-" * 20)
            
            elif opcion == "2":
                self.iniciar_sesion()
            elif opcion == "3":
                usuario = input("Ingrese su nombre de usuario: ")
                peticion = input("1-->Si quieres generar una contraseña\n 2-->Si prefieres introducir una contraseña: ")
                if peticion == "1":
                    contraseña = generar_contraseña()
                    print(contraseña)
                    hashed_password = hashlib.sha256(contraseña.encode('utf-8')).hexdigest()
                    hora_registro = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
                            # Crear un diccionario con los datos del usuario
                    nuevo_usuario = {
                        "usuario": usuario,
                        "contraseña_hash": hashed_password,
                        "hora_registro": hora_registro} 
            
                    try:
                        with open('usuarios.json', 'r') as f:
                            usuarios = json.load(f)
                    except FileNotFoundError:
                            usuarios = []
    
                    for user in usuarios:
                        if user['usuario'] == usuario:
                            print("El usuario ya existe.")
                            return
    
                    
                    usuarios.append(nuevo_usuario)
                    
                    with open('usuarios.json', 'w') as f:
                        json.dump(usuarios, f, indent=2)
        
                    print("Usuario registrado exitosamente")
                    break
                    
                else:
                    intentos = 0  # Inicializar el contador de intentos
                    max_intentos = 3
                    while intentos < max_intentos:
                        contraseña = input("Ingrese su contraseña: ")
                        if self.seguridad.comprobar_contraseña(contraseña):
                            print("La contraseña es segura")
                            hashed_password = hashlib.sha256(contraseña.encode('utf-8')).hexdigest()
                            hora_registro = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
                            # Crear un diccionario con los datos del usuario
                            nuevo_usuario = {
                                "usuario": usuario,
                                "contraseña_hash": hashed_password,
                                "hora_registro": hora_registro}
                
                            # Cargar los usuarios existentes desde el archivo JSON (si existe)
                            try:
                                with open('usuarios.json', 'r') as f:
                                    usuarios = json.load(f)
                            except FileNotFoundError:
                                usuarios = []
            
                            usuarios.append(nuevo_usuario)
                            
                            with open('usuarios.json', 'w') as f:
                                json.dump(usuarios, f, indent=2)
                
                            print("Usuario registrado exitosamente")
                            break
                            
                        else:
                            print("La contraseña no es segura, repitelo de nuevo")
                            intentos += 1
                    if intentos == max_intentos:
                        print('Has agotado los intentos')
            
    
            elif opcion == "4":
                print('Saliendo del sistema...')
                break
            else:
                print("Opción inválida. Por favor, seleccione una opción válida.")



    def menu_principal(self, usuario):

        while True:
            print('¿Qué desea realizar?')
            print('1-Reservar libro\n2-Anular una reserva\n3-Ver mis reservas\n4-Salir')
            try:
                opcion_reserva = int(input('Seleccione una opción:'))
                if opcion_reserva == 1:
                    libro_a_reservar = input("Ingrese el título del libro a reservar:")
                    self.reservar_libro(usuario, libro_a_reservar)
                elif opcion_reserva == 2:
                    titulo_a_cancelar = input("Ingrese el título del libro a cancelar: ")
                    self.anular_reserva(usuario, titulo_a_cancelar)
                    pass
                elif opcion_reserva == 3:
                    self.ver_mis_reservas(usuario)
                    pass
                elif opcion_reserva == 4:
                    print("Saliendo...")
                    break
                else:
                    print("Opción inválida. Por favor, seleccione una opción válida.")
            except ValueError:
                print("Por favor, ingrese un número válido.")
    
        else:
            print("Inicio de sesión fallido.")

def generar_contraseña(): #<--Generar la contraseña segura
    base = minus + mayus + numeros + simbolos
    longitud = 12
    
    muestra = random.sample(base, longitud)
    password = "".join(muestra)
    return password


class Seguridad: #<-- comprobar la seguridad de la contraseña introducida
    def comprobar_contraseña(self, contraseña):
        largo = len(contraseña) > 8
        mayus = any(c.isupper() for c in contraseña)
        numer = any(c.isdigit() for c in contraseña)
        return largo and mayus and numer
        pass
        




biblioteca = Biblioteca("Libros.json", "reservas.json")
biblioteca.eleccion_menu()


Biblioteca Los Hiperbóreos, ¿En que podemos ayudarle?
1--> Ver libros disponibles
2--> Inicio de Sesión:
3--> Registrarse
4--> Salir


Seleccione una opción:  1


Título: Herbert West: reanimador
Autor: H. P. Lovecraft
--------------------
Título: Risa roja
Autor: Leonid Andréiev
--------------------
Título: Ana, la de Avonlea
Autor: Lucy Maud Montgomery
--------------------
Título: Un millar de muertes
Autor: Jack London
--------------------
Título: Un par de medias de seda
Autor: Kate Chopin
--------------------
Título: El principito
Autor: Antoine de Saint-Exupéry
--------------------
Título: El hijo del lobo
Autor: Jack London
--------------------
Título: Orestes
Autor: Eurípides
--------------------
Título: Alucinación
Autor: Carmen de Burgos
--------------------
Título: Semblanzas literarias
Autor: Armando Palacio Valdés
--------------------
Título: Diario de amor
Autor: Gertrudis Gómez de Avellaneda
--------------------
Título: Hipólito
Autor: Eurípides
--------------------
Título: Kamasutra, el arte de amar
Autor: Anónimo
--------------------
Título: Los 120 días de Sodoma
Autor: Marqués de Sade
--------------------
Título: El libro de E

Seleccione una opción:  2
Ingrese su nombre de usuario:  ivan
Ingrese su contraseña:  Patrocinador1@


Inicio de sesión exitoso.
¿Qué desea realizar?
1-Reservar libro
2-Anular una reserva
3-Ver mis reservas
4-Salir


...

Reservas de ivan:
- Libro: El arte de la guerra
  Fecha de reserva: 13 de January de 2025
  Fecha de devolución: 28 de January de 2025
Si se pasa de la fecha de devolución, se le aplicará una multa de 10 euros. Gracias
¿Qué desea realizar?
1-Reservar libro
2-Anular una reserva
3-Ver mis reservas
4-Salir


Seleccione una opción: 4


Saliendo...
Biblioteca Los Hiperbóreos, ¿En que podemos ayudarle?
1--> Ver libros disponibles
2--> Inicio de Sesión:
3--> Registrarse
4--> Salir


Seleccione una opción:  3
Ingrese su nombre de usuario:  Juan10
1-->Si quieres generar una contraseña
 2-->Si prefieres introducir una contraseña:  1


G[)n:}7vj(ªP
Usuario registrado exitosamente
